In [13]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from dataclasses import dataclass
import math
from typing import Tuple, Optional
#Comment out the next line if it doesn't run
%matplotlib widget 


In [14]:
# Constants 
h  = 6.62607015e-34       # Planck constant [J s]
c  = 2.99792458e8          # Speed of light [m/s]
kB = 1.380649e-23        # Boltzmann constant [J/K]

# Astronomical conveniences
R_sun = 6.957e8          # Solar radius [m]
AU = 1.495978707e11      # Astronomical Unit [m]

### Planck BlackBody Law

In [15]:
def planckBlambda(wavelength_m, T):
    """
    Planck's spectral radiance B(lambda) of blackbody per unit wavelength.
    Unit of B in W / (m^2 sr m). Wavelength in meters, T in Kelvin.
    """
    lamd = np.asarray(wavelength_m, dtype=float)
    T = float(T)
    if T <= 0:
        raise ValueError("T must be > 0 K.")

    # Avoid lambda = 0 to prevent division by zero
    lamd = np.clip(lamd, 1e-20, None)

    # left part 
    a = 2 * h * c**2 / lamd**5

    # exponential part
    b = h * c / (lamd * kB * T)
    
    with np.errstate(over='ignore', under='ignore', invalid='ignore', divide='ignore'):
        return a / np.expm1(b)

In [16]:
planckBlambda(550e-9, 6000) # I havent check if the result is exactly correct 

np.float64(30634176630914.48)

In [17]:
def irradiance_planck(wavelength_m, R, D, T): 
    """
    Irradiance at distance D from a uniform blackbody sphere of radius R and temperature T. Other name of Flux
    Units: W / m^2 supposedly. Wavelength, Radius, Distance in meters, T in Kelvin.
    """
    F_lambda = np.pi * (R / D)** 2 * planckBlambda(wavelength_m, T)
    return F_lambda

### Distribution Function
Using Cummulative dist func because cdf is pretty much easier if we want to sample part/range of the whole dist func. We want to map random numbers to wavelengths based on the distribution. Then we sample and "create" photons randomlyh from that distribution.

In [18]:
def build_cdf_from_pdf(wavelength_m, pdf):
    """
    Build a cummulative dist. func. from certain proba. dist. func., normalized to [0, 1]. 
    The CDF is used for sampling wavelengths based on the given distribution.
    
    returns:
    wavelength_m, which is the input wavelength in meters
    cdf_vals which is the values corresponding to each wavelength, normalized to [0, 1]

    If the total probability is zero or negative, defaults to a uniform distribution.
    """
    dx = np.diff(wavelength_m)
    mid = 0.5 * (pdf[:-1] + pdf[1:]) * dx
    
    cdf_vals = np.concatenate([[0.0], np.cumsum(mid)])
    total = cdfvals[-1]
    
    if total <= 0.0:
        cdf_vals = np.linspace(0.0, 1.0, len(wavelength_m))  # if the pdf is apparently 0, we turn to linear/uniform dist
    else:
        cdf_vals = cdf_vals / total     # normalize
        
    return wavelength_m, cdf_vals

In [19]:
def sample_from_cdf(cdf_x, cdf_vals, n):
    """
    Sample n wavelengths from the CDF using inverse transform sampling.

    cdf_x = the x-grid, a.k.a wavelength in meters. Same length as cdf_vals. Must be sorted ascending.
    cdf_vals = the cdf we have (from the function), evaluated on cdf_x. Must start at ~0 and end at 1 (monotone non-decreasing).
    n = how many random samples we want.

    return:
    samples, ndarray, shape (n,)
        Samples distributed according to the CDF over cdf_x (same units as cdf_x).
    """

    u = np.random.random(size=n)                  # uniform [0,1]
    idx = np.searchsorted(cdf_vals, u, 'right')   # bin index for each u
    idx = np.clip(idx, 1, len(cdf_vals) - 1)      # keep inside valid range

    x0 = cdf_x[idx - 1]
    x1 = cdf_x[idx]
    
    y0 = cdf_vals[idx - 1]
    y1 = cdf_vals[idx]

    # linear interpolation within the CDF bin
    t = np.where(y1 > y0, (u - y0) / (y1 - y0), 0.0)
    return x0 + t * (x1 - x0)